### Histogram 1D

In [ ]:
import numpy as np
import time
import asyncio as aio
import pandas as pd
from vega.widget import VegaWidget
nbins = 50
spec_no_data = {
    '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
     'config': {'view': {'continuousHeight': 300, 'continuousWidth': 400}},
     'data': {'name': 'data'},
     'encoding': {
                  'x': {'field': 'x', 'type': 'ordinal'},
                  'y': {'field': 'y', 'type': 'quantitative'}
             },
     'mark': 'bar'}
spec_with_data = spec_no_data.copy()
nsamples = 100_000
steps = 1_000
rdata = np.random.normal(loc=0, scale=0.1, size=nsamples)
widget = VegaWidget(spec=spec_no_data)
display(widget)
async def progressive_loop(widget, nbins=50):
    hist = np.zeros(nbins)
    for sl in np.split(rdata, steps):
        await aio.sleep(0.1)
        h, _ = np.histogram(sl, bins=nbins, range=(-1.0,1.0))
        hist += h
        source = pd.DataFrame({
            'x': range(nbins),
            'y': hist,
        }, dtype='float32')
        #  widget.update('data', remove='datum.x>=0')
        #print("displayed", widget._displayed)
        widget._displayed = True
        widget.update_dataframe(source, remove="true")

await progressive_loop(widget)

### Histogram 2D

In [ ]:
import numpy as np
import asyncio as aio
import pandas as pd
from vega.widget import VegaWidget
nbins = 128
spec_no_data = {'$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
 'config': {'view': {'continuousHeight': 300, 'continuousWidth': 400}},
 'data': {'name': 'data'},
 'encoding': {'color': {'field': 'z', 'type': 'quantitative'},
  'x': {'field': 'x', 'type': 'ordinal', "bin": {"maxbins":60}},
  'y': {'field': 'y', 'type': 'ordinal', "bin": {"maxbins":60}}},
 'mark': 'rect'}
widget = VegaWidget(spec=spec_no_data)
display(widget)
nsamples=100_1000
steps = 1000
mean = [0.1, 0.3]
cov = [[0.01, 0], [0, 0.09]]
rdata = np.random.multivariate_normal(mean, cov, size=(nsamples))
async def progressive_loop(widget, nbins=128):
    hist = np.zeros((nbins,nbins), dtype='float32')
    for sl in np.split(rdata, steps):
        await aio.sleep(0.1)
        h, *_ = np.histogram2d(sl[0],sl[1], bins=nbins, range=((-1.0,1.0),(-1.0,1.0)))
        hist += h
        #print(hist.shape)
        widget._displayed = True
        widget.update_histogram2d(hist, ['x', 'y', 'z'], remove="true")

await progressive_loop(widget)